In [1]:
import requests
import pandas as pd
import zipfile
import io

# Miasta

In [2]:
url = 'https://github.com/PawelJasinski25/Wizualizacja/raw/main/czyszczenie%20i%20poprawa%20danych/miasta-output.zip'

response = requests.get(url)
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        print(z.namelist())
        with z.open('output2.csv') as f:
            df = pd.read_csv(f)
else:
    print(f'Failed to download file: {response.status_code}')

['output2.csv']


Dane zawierały m.in obszary wiejskie, dzielnice Warszawy. Dzięki filtrowaniu po ostatniej cyfrze kodu uzyskujemy same miasta

In [4]:
df['cityId'] = df['cityId'].astype(str)

# Utwórz nowy DataFrame, który będzie zawierał tylko wiersze z 'cityId' kończącymi się na '1' lub '4'
filtered_df = df[df['cityId'].str.endswith(('1', '4'))].copy()
filtered_df.head()


,id,categoryId,categoryName,cityId,cityName,year,value
4060,1683,1230,Biblioteki publiczne,71427103144,Żelechów - miasto,1995,1
4061,1684,1230,Biblioteki publiczne,71427103144,Żelechów - miasto,1996,1
4062,1685,1230,Biblioteki publiczne,71427103144,Żelechów - miasto,1997,1
4063,1686,1230,Biblioteki publiczne,71427103144,Żelechów - miasto,1998,1
4064,1687,1230,Biblioteki publiczne,71427103144,Żelechów - miasto,1999,1


# Usuwanie przyrostków

In [5]:
filtered_df['cityName'] = filtered_df['cityName'].str.replace(' - miasto', '', regex=False)
filtered_df.head()

,id,categoryId,categoryName,cityId,cityName,year,value
4060,1683,1230,Biblioteki publiczne,71427103144,Żelechów,1995,1
4061,1684,1230,Biblioteki publiczne,71427103144,Żelechów,1996,1
4062,1685,1230,Biblioteki publiczne,71427103144,Żelechów,1997,1
4063,1686,1230,Biblioteki publiczne,71427103144,Żelechów,1998,1
4064,1687,1230,Biblioteki publiczne,71427103144,Żelechów,1999,1


# Wybranie miast powyżej 30k

In [6]:
filtered_df['value'] = pd.to_numeric(filtered_df['value'].replace('-', float('nan')), errors='coerce')
cities_over_30k = filtered_df[(filtered_df['categoryName'] == 'ludność w tysiącach') & (filtered_df['value'] > 30)]
cities_over_30k.head()


,id,categoryId,categoryName,cityId,cityName,year,value
142854,57443,1645341,ludność w tysiącach,52615261011,Kielce,1995,213.78
142855,57444,1645341,ludność w tysiącach,52615261011,Kielce,1996,213.75
142856,57445,1645341,ludność w tysiącach,52615261011,Kielce,1997,212.60
142857,57446,1645341,ludność w tysiącach,52615261011,Kielce,1998,212.38
142858,57447,1645341,ludność w tysiącach,52615261011,Kielce,1999,214.24


In [7]:
city_ids_over_30k = cities_over_30k['cityId'].unique()
filtered_df_selected = filtered_df[filtered_df['cityId'].isin(city_ids_over_30k)]

filtered_df_selected.head()

,id,categoryId,categoryName,cityId,cityName,year,value
141868,57044,1230,Biblioteki publiczne,52615261011,Kielce,1995,17.0
141869,57045,1230,Biblioteki publiczne,52615261011,Kielce,1996,17.0
141870,57046,1230,Biblioteki publiczne,52615261011,Kielce,1997,15.0
141871,57047,1230,Biblioteki publiczne,52615261011,Kielce,1998,15.0
141872,57048,1230,Biblioteki publiczne,52615261011,Kielce,1999,15.0


In [8]:
filtered_df_selected.to_csv('over_30k.csv', index=False)

# Obliczenie średniej zmiany liczby ludności na1000 mieszkańców dla każdego miasta

In [9]:
change = filtered_df_selected[(filtered_df_selected['categoryName'] == 'zmiana liczby ludności na 1000 mieszkańców')]

In [10]:
change.head()

,id,categoryId,categoryName,cityId,cityName,year,value
142332,57329,458603,zmiana liczby ludności na 1000 mieszkańców,52615261011,Kielce,2003,-5.5
142333,57330,458603,zmiana liczby ludności na 1000 mieszkańców,52615261011,Kielce,2004,-5.6
142334,57331,458603,zmiana liczby ludności na 1000 mieszkańców,52615261011,Kielce,2005,-6.0
142335,57332,458603,zmiana liczby ludności na 1000 mieszkańców,52615261011,Kielce,2006,-4.8
142336,57333,458603,zmiana liczby ludności na 1000 mieszkańców,52615261011,Kielce,2007,-6.2


In [11]:
change = change[change['value'].notnull()]
average_change = change.groupby(['cityId', 'cityName'])['value'].mean().reset_index()

In [12]:
average_change.head()

,cityId,cityName,value
0,11212161011,Kraków,1.728571
1,11212262011,Nowy Sącz,-1.104762
2,11212303034,Chrzanów,-7.442857
3,11212312054,Olkusz,-6.276190
4,11212463011,Tarnów,-6.614286


In [13]:
average_change_sorted = average_change.sort_values(by='value', ascending=False)

In [14]:
average_change_sorted.head(10)

,cityId,cityName,value
97,71412934021,Marki,30.147619
98,71412934031,Ząbki,28.704762
101,71413018044,Piaseczno,17.590476
100,71413005044,Grodzisk Mazowiecki,12.142857
44,23016121011,Luboń,11.680952
88,61813563011,Rzeszów,10.690476
39,20811462011,Zielona Góra,8.485714
66,42214015021,Rumia,7.033333
102,71413021021,Pruszków,6.733333
35,12415108021,Mikołów,3.747619


In [15]:
average_change_sorted.to_csv('average_change.csv', index=False)

# Powiaty

In [16]:
url = 'https://github.com/PawelJasinski25/Wizualizacja/raw/main/czyszczenie%20i%20poprawa%20danych/powiaty-output.zip'

response = requests.get(url)
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        print(z.namelist())
        with z.open('powiaty-output.csv') as f:
            df2 = pd.read_csv(f)
else:
    print(f'Failed to download file: {response.status_code}')

['powiaty-output.csv']


# Usunięcie przedrostków

Powiaty które dotyczyły miast były w takiej formule:
<br>Powiat m.Kraków
<br>Powiat m.st. Warszawa



In [17]:
df2['value'] = pd.to_numeric(df2['value'].replace('-', float('nan')), errors='coerce')
df2['powiatName'] = df2['powiatName'].str.replace(r'm\. st\.? |m\. ', '', regex=True)
df2.to_csv('powiaty.csv', index=False)